# **pysummarization & bert-extractive-summarizer‗English‗string**
- pysummarizationは、自然言語処理とニューラルネットワーク言語モデルを用いたPythonのテキスト自動要約ライブラリ。
- テキストクラスタリングによって元の文書やウェブスクレイピングされたテキストの主要なポイントをまとめた要約が作成できる。Accel-brain-baseを用いて、LSTMをベースとしたEncoder/Decoderを実装し、Sequence-to-Sequence（Seq2Seq）学習により要約精度を向上させているようだ。
- bert-extractive-summarizerは、HuggingFace Pytorch transformersライブラリを利用して、抽出的な要約を実行します。 これは、最初に文を埋め込み、次にクラスタリングアルゴリズムを実行して、クラスターの重心に最も近い文を見つけることで機能します。

---
- これは <font color= "LightCoral">**英文要約専用**</font> です。
- **「ランタイム→すべてのセルを実行」**し、ライブラリが読み込まれた後、表示される**[ファイル選択] ボタンをクリックし、要約したいテキストファイル（.txt）指定するだけ**です。
- **要約後、result_summary.txt と result_summary.docx を生成**します。[**DeepL翻訳**](https://www.deepl.com/ja/translator) はdocxデータをドラックするだけで翻訳できます。
- **要約結果**に「source」「result_summary」「result summary:SL」「	bert_extra_summary」が表示されます。
- SF（SimilarityFilter｜類似性フィルター）：文章内にある文字列に対し類似性の尺度を使って計算し冗長な文章を短くまとめるpysummarizationの機能。カットオフ設定はスライドバーにて設定できます。※設定幅：0.05～0.5（Default=0.25, 0.05Step）
- Specified with ratio setting in bert-extractive-summarizer：bert-extractive-summarizerにおける要約ratioの設定です。※設定幅：0.05～0.5（Default=0.2, step:0.05）
- <font color= "LightCoral">**[注意]**</font> テキストファイルは文字コードを「UTF-8」としてください。


# **ライブラリインストール & インポート**

In [1]:
pip install pysummarization

     |████████████████████████████████| 51 kB 268 kB/s 
  Created wheel for pysummarization: filename=pysummarization-1.1.8-py3-none-any.whl size=59453 sha256=bde6044382b66d32ef8f479af800684a6039e9ec090ef58a61f5d9756d3f354e
  Stored in directory: /root/.cache/pip/wheels/da/14/3e/02d15001af23ca877c5149b66280a605e5cdbbe76972598afa
Successfully built pysummarization


In [2]:
#日本語フォントをインストール
#!apt-get -y install fonts-ipafont-gothic

In [3]:
#Mecabのインストール
#!pip install mecab-python3==0.996.5

In [4]:
pip install bert-extractive-summarizer

     |████████████████████████████████| 3.1 MB 8.9 MB/s 
     |████████████████████████████████| 895 kB 57.8 MB/s 
     |████████████████████████████████| 596 kB 61.2 MB/s 
     |████████████████████████████████| 3.3 MB 42.4 MB/s 
     |████████████████████████████████| 59 kB 7.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
pip install python-docx

     |████████████████████████████████| 5.6 MB 6.9 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184508 sha256=f0ea7e9e03e6be41ecfbc9c6b0e08eebd392eb9695371b775cff0532f506636a
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [6]:
import pandas as pd
import re
from docx import Document
from docx.shared import RGBColor
from docx.shared import Inches
from docx.shared import Pt

In [7]:
from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor
from pysummarization.nlp_base import NlpBase
from pysummarization.similarityfilter.tfidf_cosine import TfIdfCosine
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer

# **テキスト読込み**

In [8]:
from google.colab import files
print('txtファイル(UTF-8）を指定してください')
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

txtファイル(UTF-8）を指定してください


Saving trump.txt to trump.txt
User uploaded file "trump.txt" with length 30500 bytes


In [9]:
#@title Similarity_limit setting in pysummarization（Defalt = 0.25） { run: "auto" }
similarity_limit = 0.25 #@param {type:"slider", min:0.05, max:0.5, step:0.05}
#similarity_filter.similarity_limit = 0.25

In [21]:
#@title Specified with ratio setting in bert-extractive-summarizer（Defalt = 0.2） { run: "auto" }
ratio_set = 0.2 #@param {type:"slider", min:0.05, max:0.5, step:0.05}

# **テキスト要約処理**

In [11]:
if len(uploaded.keys()) != 1:
    print("アップロードは１ファイルにのみ限ります")
else:
    target = list(uploaded.keys())[0]
    
with open(target) as f:
    contents = f.readlines()

document = ''.join(contents)

In [12]:
print(u'[source]')
print(document)

# Object of automatic summarization.
auto_abstractor = AutoAbstractor()
# Set tokenizer.
auto_abstractor.tokenizable_doc = SimpleTokenizer()
# Set delimiter for making a list of sentence.
auto_abstractor.delimiter_list = [".", "\n"]
# Object of abstracting and filtering document.
abstractable_doc = TopNRankAbstractor()
# Summarize document.
result_dict1 = auto_abstractor.summarize(document, abstractable_doc)

# Output result.
print(u'[result summary]')
for sentence in result_dict1["summarize_result"]:
    print(sentence)

# NLPオブジェクト
nlp_base = NlpBase()
# トークナイザー設定
nlp_base.tokenizable_doc = SimpleTokenizer()
# 類似性フィルター
similarity_filter = TfIdfCosine()
# NLPオブジェクト設定
similarity_filter.nlp_base = nlp_base
# 類似性limit：limit超える文は切り捨て
similarity_filter.similarity_limit = similarity_limit

# Object of automatic summarization.
auto_abstractor = AutoAbstractor()
# Set tokenizer.
auto_abstractor.tokenizable_doc = SimpleTokenizer()
# Set delimiter for making a list of sentence.
auto_abstractor.delimiter_list = [".", "\n"]
# Object of abstracting and filtering document.
abstractable_doc = TopNRankAbstractor()
# Summarize document.
result_dict2 = auto_abstractor.summarize(document, abstractable_doc, similarity_filter)

# Output result.
print(u'[result summary:SL]')
for sentence in result_dict2["summarize_result"]:
    print(sentence)

[source]
Friends, delegates and fellow Americans: I humbly and gratefully accept your nomination for the presidency of the United States.

Who would have believed that when we started this journey on June 16th of last year, we – and I say we because we are a team – would have received almost 14 million votes, the most in the history of the Republican Party, and that the Republican Party would get 60 percent more votes than it received eight years ago. Who would have believed this? Who would have believed this? The Democrats, on the other hand, received 20 percent fewer votes than they got four years ago. Not so good. Not so good.

Together, we will lead our party back to the White House, and we will lead our country back to safety, prosperity, and peace. We will be a country of generosity and warmth. But we will also be a country of law and order.

Our Convention occurs at a moment of crisis for our nation. The attacks on our police, and the terrorism in our cities, threaten our very w

/usr/local/lib/python3.7/dist-packages/pysummarization/similarityfilter/tfidf_cosine.py:46: RuntimeWarning: invalid value encountered in double_scalars
  result = dot_prod / (norm_x * norm_y)


[result summary:SL]
Friends, delegates and fellow Americans: I humbly and gratefully accept your nomination for the presidency of the United States.

Who would have believed that when we started this journey on June 16th of last year, we – and I say we because we are a team – would have received almost 14 million votes, the most in the history of the Republican Party, and that the Republican Party would get 60 percent more votes than it received eight years ago.

 We will be a country of generosity and warmth.

 But we will also be a country of law and order.

I have a message for all of you: the crime and violence that today afflicts our nation will soon, and I mean very soon, come to an end.

Again, I will tell you the plain facts that have been edited out of your nightly news and your morning newspaper: Nearly Four in 10 African-American children are living in poverty, while 58% of African-American youth are now not employed.

This was just prior to the signing of the Iran deal, whi

In [13]:
# bert-extractive-summarizer
from summarizer import Summarizer

model = Summarizer()
result = model(document, ratio=ratio_set)  # Specified with ratio
#result = model(document, num_sentences=10)  # Will return 3 sentences 
#result = model(document, min_length=60)
full = ''.join(result)
print(full)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Friends, delegates and fellow Americans: I humbly and gratefully accept your nomination for the presidency of the United States. Who would have believed that when we started this journey on June 16th of last year, we – and I say we because we are a team – would have received almost 14 million votes, the most in the history of the Republican Party, and that the Republican Party would get 60 percent more votes than it received eight years ago. We will be a country of generosity and warmth. But we will also be a country of law and order. So if you want to hear the corporate spin, the carefully-crafted lies, and the media myths the Democrats are holding their convention next week. And almost 4,000 have been killed in the Chicago area since he took office. They are being released by the tens of thousands into our communities with no regard for the impact on public safety or resources. Again, I will tell you the plain facts that have been edited out of your nightly news and your morning news

In [14]:
doc0 = ''.join(s for s in document)
doc1 = result_dict1["summarize_result"]
doc2 = result_dict2["summarize_result"]
doc1 = ''.join(s for s in doc1)
doc2 = ''.join(s for s in doc2)
doc3 = ''.join(s for s in full)
lst1 = ["source","result summary","result summary:SL","bert_extra_summary"]
lst2 = [doc0,doc1,doc2,doc3]
df = pd.DataFrame(list(zip(lst1,lst2)), columns = ['Class.','Content'])
df = df.replace( '\n', '', regex=True)

In [15]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    df = df.stack().str.lstrip().unstack()
    df = df.style.set_properties(**{'text-align': 'left'})

In [16]:
#要約結果出力（txt & docx)
result = '[result_summary]' +'\n'+ doc1 +'\n'+ '[result_summary:similarity_limit]' +'\n'+ doc2 + '[result_summary:bert_extra]' +'\n'+ doc3
with open("result_summary.txt", "w") as text_file:
    text_file.write("String Variable: %s" % result)

docx = Document() 
docx.add_paragraph(result)
docx.save('result_summary.docx')

# **テキスト要約結果**

In [17]:
df

# **備考**
- [**【自然言語処理（日本語）】自動要約ライブラリのpysummarizationを使う為の準備と実装**](https://zerofromlight.com/blogs/detail/35/)
- [**トランプ氏受諾演説（英語全文）2016年7月23日**](https://www3.nhk.or.jp/news/special/2016-presidential-election/republic4.html)
- [**bert-extractive-summarizer 0.8.1**](https://pypi.org/project/bert-extractive-summarizer/)
- [**【python】 bert-extractive-summarizerを使った文章要約**](https://kagglenote.com/ml-tips/bert-extractive-summarizer/)